In [1]:
from flask import Flask, render_template, request, jsonify
import pandas as pd
import numpy as np
import pickle
import nest_asyncio

nest_asyncio.apply()  # Allow Flask to run inside Jupyter

app = Flask(__name__)

# Load trained model
with open("model.pkl", "rb") as f:
    model = pickle.load(f)

# Load individual label encoders
encoders = {}
for feature in ["make", "model", "body", "state", "color", "interior"]:
    with open(f"{feature}_encoder.pkl", "rb") as f:
        encoders[feature] = pickle.load(f)

# Load data
df = pd.read_csv("car_prices_processed.csv")

# Get unique dropdown options
makes = sorted(df["make"].unique().tolist(), reverse=False)
bodies = df["body"].unique().tolist()
states = sorted(df["state"].unique().tolist(), reverse=False)
colors = df["color"].unique().tolist()
interiors = df["interior"].unique().tolist()
years = sorted(df["year"].unique().tolist(), reverse=True)

# Create a mapping of makes -> models
make_body_model_map = {}
for make in makes:
    make_body_model_map[make] = df[df["make"] == make].groupby("body")["model"].unique().apply(list).to_dict()

In [2]:
@app.route("/", methods=["GET", "POST"])
def index():
    predicted_price = None
    selected_year = None
    selected_make = None
    selected_model = None
    selected_body = None
    selected_state = None
    selected_odometer = None
    selected_color = None
    selected_interior = None
    selected_newcondition = None
    selected_transmission = None

    if request.method == "POST":
        # Get user input
        selected_year = int(request.form["year"])
        selected_make = request.form["make"]
        selected_model = request.form["model"]
        selected_body = request.form["body"]
        selected_state = request.form["state"]
        selected_odometer = float(request.form["odometer"])
        selected_color = request.form["color"]
        selected_interior = request.form["interior"]
        selected_newcondition = float(request.form["newcondition"])
        selected_transmission = request.form["transmission"]

        # Encode categorical inputs
        make_encoded = encoders["make"].transform([selected_make])[0]
        model_encoded = encoders["model"].transform([selected_model])[0]
        body_encoded = encoders["body"].transform([selected_body])[0]
        state_encoded = encoders["state"].transform([selected_state])[0]
        color_encoded = encoders["color"].transform([selected_color])[0]
        interior_encoded = encoders["interior"].transform([selected_interior])[0]

        # Prepare input for model
        input_data = np.array([[selected_year, make_encoded, model_encoded, body_encoded, state_encoded,
                                selected_odometer, color_encoded, interior_encoded, selected_newcondition,
                                1 if selected_transmission == "Manual" else 0]])

        # Predict price
        predicted_price = model.predict(input_data)[0]

    return render_template("model_page.html", makes=makes, bodies=bodies, states=states, colors=colors, 
                           interiors=interiors, years=years, predicted_price=predicted_price, 
                           make_body_model_map=make_body_model_map,
                           selected_year=selected_year, selected_make=selected_make, selected_model=selected_model,
                           selected_body=selected_body, selected_state=selected_state, 
                           selected_odometer=selected_odometer, selected_color=selected_color,
                           selected_interior=selected_interior, selected_newcondition=selected_newcondition,
                           selected_transmission=selected_transmission)


# AJAX route to get models based on selected make and body
@app.route("/get_models/<make>/<body>")
def get_models(make, body):
    models = make_body_model_map.get(make, {}).get(body, [])
    return jsonify(models)

In [ ]:
from werkzeug.serving import run_simple

run_simple("localhost", 5000, app)

 * Running on http://localhost:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Mar/2025 01:31:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2025 01:31:59] "GET /get_models/Bentley/Suv HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2025 01:32:01] "GET /get_models/Bentley/Sedan HTTP/1.1" 200 -
C:\Users\misha\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
127.0.0.1 - - [09/Mar/2025 01:32:21] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2025 01:32:21] "GET /favicon.ico HTTP/1.1" 404 -
